In [3]:
ALLUXIO = False
NUMBER_OF_ROUNDS = 10
BUCKET = "BUCKET_NAME"

VBox()

In [4]:
df = spark.read.csv(f"s3://{BUCKET}/alluxio-emr/lineitem.tbl", sep="|").repartition(1000)
df.cache()
count = df.count()
print(f"Number of records: {count}")
print(f"Number of partitions: {df.rdd.getNumPartitions()}")

VBox()

Number of records: 600037902
Number of partitions: 1000

In [5]:
from time import time

path = "alluxio:///test/data/" if ALLUXIO else f"s3://{BUCKET}/alluxio-emr/data/"
write_times = []
read_times = []
df.write.parquet(path, mode="overwrite")
for _ in range(NUMBER_OF_ROUNDS):
    start = time()
    df.write.parquet(path, mode="overwrite")
    write_times.append(time() - start)
    start = time()
    df_temp = spark.read.parquet(path).cache()
    assert count == df_temp.count()
    read_times.append(time() - start)
    df_temp.unpersist()
df.unpersist()

VBox()

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string]

In [6]:
print(f"Min write time: {min(write_times):.1f} sec")
print(f"Avg write time: {(sum(write_times)/len(write_times)):.1f} sec")
print(f"Max write time: {max(write_times):.1f} sec")
print(f"Min read time: {min(read_times):.1f} sec")
print(f"Avg read time: {(sum(read_times)/len(read_times)):.1f} sec")
print(f"Max read time: {max(read_times):.1f} sec")

VBox()

Min write time: 62.2 sec
Avg write time: 67.4 sec
Max write time: 77.7 sec
Min read time: 94.3 sec
Avg read time: 97.7 sec
Max read time: 102.7 sec